In [ ]:
import tensorflow as tf
numGPUs = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", numGPUs)

In [ ]:
import random
import pandas
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D,Dropout,Activation,MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras import backend as K
import talos
import os
import math
import threading
from matplotlib import pyplot
import tensorflow_datasets as tfds
from pathlib import Path
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
numClients = 50
num_classes = 10
dataset_name = 'svhn_cropped'
experiment_name = 'svhn'
Path(experiment_name+"_res/intervals_res"+str(numClients)).mkdir(parents=True, exist_ok=True)

In [ ]:
#global parameters for grid search over activation function
degree = 3
interval = 0

def reluApprox(x):
    if degree == 3:  
        if interval == 3:
            return 0.7146 + 1.5000*K.pow(x/interval,1)+0.8793*K.pow(x/interval,2)
        if interval == 5:
            return 0.7865 + 2.5000*K.pow(x/interval,1)+1.88*K.pow(x/interval,2)
        if interval == 7:
            return 0.9003 + 3.5000*K.pow(x/interval,1)+2.9013*K.pow(x/interval,2)
        if interval == 10:
            return 1.1155 + 5*K.pow(x/interval,1)+4.4003*K.pow(x/interval,2)
        if interval == 12:
            return 1.2751 + 6*K.pow(x/interval,1)+5.3803*K.pow(x/interval,2)
    if degree == 5:  
        if interval == 7:
            return 0.7521 + 3.5000*K.pow(x/interval,1)+4.3825*K.pow(x/interval,2)-1.7281*K.pow(x/interval,4)
        if interval == 20:
            return 1.3127 + 10*K.pow(x/interval,1)+15.7631*K.pow(x/interval,2)-7.6296*K.pow(x/interval,4)
def sigmoidApprox(x):
    if degree == 3:  
        if interval == 3:
            return 0.5 + 0.6997*K.pow(x/interval,1)-0.2649*K.pow(x/interval,3)
        if interval == 5:
            return 0.5 + 0.9917*K.pow(x/interval,1)-0.5592*K.pow(x/interval,3)
        if interval == 7:
            return 0.5 + 1.1511*K.pow(x/interval,1)-0.7517*K.pow(x/interval,3)
        if interval == 8:
            return 0.5 + 1.2010*K.pow(x/interval,1)-0.8156*K.pow(x/interval,2)
        if interval == 12:
            return 0.5 + 1.2384*K.pow(x/interval,1)-0.8647*K.pow(x/interval,2)
def tanApprox(x):
    if degree == 3: 
        if interval == 1:
            return 0.9797*K.pow(x/interval,1)-0.2268*K.pow(x/interval,3)        
        if interval == 2:
            return 1.7329*K.pow(x/interval,1)-0.8454*K.pow(x/interval,3)
        if interval == 3:
            return 2.1673*K.pow(x/interval,1)-1.3358*K.pow(x/interval,3)
        if interval == 5:
            return 2.5338*K.pow(x/interval,1)-1.8051*K.pow(x/interval,3)
        if interval == 7:
            return 2.6629*K.pow(x/interval,1) -1.9801*K.pow(x/interval,3)
        if interval == 12:
            return 2.7599*K.pow(x/interval,1)-2.1140*K.pow(x/interval,2)
    if degree == 12:
        print('ooopssss')

In [ ]:
# Set intervals to search, check intervals in the approx function
intervals = [1,2,3,5,7,12]

# Set approximated activation function and best parameters
approx_act_fn = [tanApprox]
intervals_params = dict(learn_rate=[0.193],
                        batch_size=[8],
                        momentum=[0.776],
                        epochs=[29], 
                        act_fn=approx_act_fn)

In [ ]:
def get_model(params):
    model = Sequential()
    model.add(Conv2D(6, (5,5), activation=params['act_fn'], input_shape=input_shape,name="1"))
    model.add(AveragePooling2D((2,2),name="2"))
    model.add(Conv2D(16, (5,5), activation=params['act_fn'],name="3"))
    model.add(AveragePooling2D((2,2),name="4"))
    model.add(Flatten(name="5"))
    model.add(Dense(120, activation=params['act_fn'],name="6"))
    model.add(Dense(84, activation=params['act_fn'],name="7"))
    model.add(Dense(num_classes, activation='softmax',name="f"))
    
    return model

In [ ]:
# Model / data parameters
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = tfds.as_numpy(tfds.load(dataset_name,
                                                               split=['train','test'],
                                                               batch_size=-1,
                                                               as_supervised=True))

# Scale images to the [0, 1] range
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train = x_train / 255
x_test = x_test / 255
# Make sure images have shape (28, 28, 1)
print(x_train.shape)
print(x_test.shape)
#x_train = np.expand_dims(x_train, -1)
#x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")
y_test_uncat = y_test

#to use with mean sq error
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#shuffle both train and test once (to average between runs later on..)
shuffler = np.random.permutation(len(x_train))
x_train = x_train[shuffler]
y_train = y_train[shuffler]
shuffler = np.random.permutation(len(x_test))
x_test = x_test[shuffler]
y_test = y_test[shuffler]

sample_height = x_train[0].shape[0]
sample_width = x_train[0].shape[1]
sample_channels = x_train[0].shape[2]


input_shape = (sample_height, sample_width, sample_channels)

def randomSplitClientsData(data,labels,numParties):
    numSamplesPerClient = int(data.shape[0]/numParties)
    print(numSamplesPerClient)
    clientsData = np.zeros((numParties,int(numSamplesPerClient),sample_height,sample_width,sample_channels))
    clientsDataLabels = np.zeros((numParties,int(numSamplesPerClient),num_classes))
    #print(numSamplesPerClient)
    ind = 0
    for i in range(numParties):
        clientsData[i] = data[ind:ind+numSamplesPerClient]
        clientsDataLabels[i]=labels[ind:ind+numSamplesPerClient]
        ind = ind+numSamplesPerClient
    return clientsData, clientsDataLabels

def prepare_data_for_X_clients(numClients):
    clientsData, clientsDataLabels = randomSplitClientsData(x_train, y_train, numClients)
    return clientsData, clientsDataLabels

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history, params):
    print('##########################################################')
    print(params)
    # plot loss
    pyplot.subplot(211)
    pyplot.title('MSE')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')
    # plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
    # save plot to file
    #filename = sys.argv[0].split('/')[-1]
    #pyplot.savefig(filename + '_plot.png')
    #pyplot.close()
    pyplot.show()
    print('##########################################################')

In [ ]:
hist = []
hist_params = []

def experiment(x_train, y_train, x_val, y_val, params):
        
    
    optimizer = SGD(learning_rate=params['learn_rate'], momentum=params['momentum'], nesterov=False, name='SGD')
    
    model = get_model(params)
    model.compile(optimizer=optimizer,
                  loss="mean_squared_error",
                  metrics=['accuracy', tf.keras.metrics.Recall(),tf.keras.metrics.Precision()],
                  run_eagerly=False)

    history = model.fit(x=x_train,
                    y=y_train,
                    epochs=params['epochs'],
                    batch_size=params['batch_size'],
                    validation_data=(x_val, y_val),
                    verbose=0)

    hist.append(history)
    hist_params.append(params)
        
    return history, model

In [ ]:
def grid_search_for_X_clients(numClients, clientsData, clientsDataLabels, param_grid):

    scan_res = np.zeros(numClients, dtype=object)
    
    def client_gridsearch(i):
        #Distribute load accross GPUs
        with tf.device('/GPU:'+str(i%numGPUs)):

            print('training in client ', i, 'interval is',interval)
            scan_results = talos.Scan(x=clientsData[i],
                                      y=clientsDataLabels[i],
                                      params=param_grid,
                                      model=experiment,
                                      experiment_name=experiment_name)
            scan_res[i]=scan_results
            
        
    # Batch multithreading
    n_batch = int(math.ceil(float(numClients)/(numGPUs)))
    print(n_batch)
    remaining_clients = numClients
    for i in range(n_batch):
        threads = list()

        for j in range(min(numGPUs, remaining_clients)):
            t = threading.Thread(target=client_gridsearch, args=(i*(numGPUs)+j,))
            threads.append(t)
            t.start()

        for _,t in enumerate(threads):
            remaining_clients -= 1
            t.join()
    
    return scan_res

In [ ]:
interval_res = list()
clientsData, clientsDataLabels = prepare_data_for_X_clients(numClients)

for i in intervals:
    global interval
    interval = i
    
    interval_res.append(grid_search_for_X_clients(numClients, clientsData, clientsDataLabels, intervals_params))

In [ ]:
best_interval = 0

choice_ratio = np.zeros(numClients, dtype=object)
choice_ratio.fill((0,0.0))

for index, i_res in enumerate(interval_res):
    
    i_res = [r for r in i_res if not r == 0]
    
    for c, client_scan in enumerate(i_res):
        if not client_scan == 0:
            curr_acc = client_scan.data.head(1)['val_accuracy'].item()

            display(client_scan.data)

            if choice_ratio[c][1] <= curr_acc:
                choice_ratio[c] = (intervals[index], curr_acc)      

            ## Write dataframes to files
            client_scan.data.to_csv(experiment_name+"_res/intervals_res"+str(numClients)+"/intervals_res_client"+str(c)+"_interval_"+str(intervals[index])+".csv")

choice_ratio = np.array([e[0] for e in choice_ratio])

res_ratio = np.zeros(len(intervals), dtype=float)

for i,e in enumerate(intervals):
    res_ratio[i] = np.count_nonzero(choice_ratio == e)

res_ratio = res_ratio / numClients

best_interval = intervals[np.argmax(res_ratio)]
print(res_ratio, best_interval)

In [ ]:
def test_metrics_for_X_clients(numClients, clientsData, clientsDataLabels, avg_test_params):

    metrics_res = np.zeros(numClients, dtype=object)
    
    
    def client_test_metrics(i):
        #Distribute load accross GPUs
        with tf.device('/GPU:'+str(i%numGPUs)):

            print('training in client ', i, 'interval is',interval)
            model = get_model(avg_test_params)
            optimizer = SGD(learning_rate=avg_test_params['learn_rate'], momentum=avg_test_params['momentum'], nesterov=False, name='SGD')

            model.compile(optimizer=optimizer,
                  loss="mean_squared_error",
                  metrics=['accuracy', tf.keras.metrics.Recall(),tf.keras.metrics.Precision()],
                  run_eagerly=False)
            model.fit(x=clientsData[i],
                        y=clientsDataLabels[i],
                        epochs=avg_test_params['epochs'],
                        batch_size=avg_test_params['batch_size'],
                        verbose=0)
            print('evaluation in client ', i)
            metrics = model.evaluate(x_test, y_test)
            metrics_res[i] = metrics
            
        
    # Batch multithreading
    n_batch = int(math.ceil(float(numClients)/(numGPUs)))
    print(n_batch)
    remaining_clients = numClients
    for i in range(n_batch):
        threads = list()

        for j in range(min(numGPUs, remaining_clients)):
            t = threading.Thread(target=client_test_metrics, args=(i*(numGPUs)+j,))
            threads.append(t)
            t.start()

        for _,t in enumerate(threads):
            remaining_clients -= 1
            t.join()
    
    return metrics_res

In [ ]:
####### Retrain each client to run the test set and get best metrics
intervals_test_res = []
interval = best_interval
intervals_params_test = dict(learn_rate=intervals_params['learn_rate'][0],
                        batch_size=intervals_params['batch_size'][0],
                        momentum=intervals_params['momentum'][0],
                        epochs=intervals_params['epochs'][0], 
                        act_fn=intervals_params['act_fn'][0])

clientsData, clientsDataLabels = prepare_data_for_X_clients(numClients)

intervals_test_res = test_metrics_for_X_clients(numClients, clientsData, clientsDataLabels, intervals_params_test)

In [ ]:
best_index = 1
intervals_test_res = [r for r in intervals_test_res if not r == 0]
for index, i_res in enumerate(intervals_test_res):
    
    if i_res[1] >= intervals_test_res[best_index][1]:
        best_index = index
            
print("Best test accuracy :", intervals_test_res[best_index][1])
print("Loss :", intervals_test_res[best_index][0])
print("Precision :", intervals_test_res[best_index][2])
print("Recall :", intervals_test_res[best_index][3])

In [ ]:
print(intervals_params_test)
print(interval)